In [ ]:
# Importamos las librerías necesarias

import cv2
import mediapipe as mp
import numpy as np

In [82]:
# Iniciamos cámara
vid = cv2.VideoCapture(0)

#Inicializamos las variables de las manos
mpHands = mp.solutions.hands
hands = mpHands.Hands()
mpDraw = mp.solutions.drawing_utils

height = 480
width = 640
drawn = np.zeros((width,height))
thicknes = np.zeros((width,height))

#Le pasamos los landmarks, la mano (derecha o izquierda) y la parte de la mano que se quiere trackear
def countFingers(handLms, hand):
    arr = np.zeros(3)
    count = 0
    #Pulgar
    if(hand == "Right"):
        if (handLms.landmark[mpHands.HandLandmark.THUMB_TIP].x < handLms.landmark[mpHands.HandLandmark.THUMB_MCP].x):
            count += 1
    else:
        if (handLms.landmark[mpHands.HandLandmark.THUMB_TIP].x > handLms.landmark[mpHands.HandLandmark.THUMB_MCP].x):
            count += 1
    
    #Índice
    if (handLms.landmark[mpHands.HandLandmark.INDEX_FINGER_TIP].y < handLms.landmark[mpHands.HandLandmark.INDEX_FINGER_PIP].y):
        count += 1
        if(hand == "Right"):
            x = int(handLms.landmark[mpHands.HandLandmark.INDEX_FINGER_TIP].x * w)
            y = int(handLms.landmark[mpHands.HandLandmark.INDEX_FINGER_TIP].y * h)
            if(x >= 640):
                x = 639
                
            if (x < 0):
                x = 0
            
            if(y >= 480):
                y=479    
                
            if(y < 0):
                y = 0
                
            arr[0] = x
            arr[1] = y
            
                
        
    #Corazón
    if (handLms.landmark[mpHands.HandLandmark.MIDDLE_FINGER_TIP].y < handLms.landmark[mpHands.HandLandmark.MIDDLE_FINGER_PIP].y):
        count += 1
        
    #Anular
    if (handLms.landmark[mpHands.HandLandmark.RING_FINGER_TIP].y < handLms.landmark[mpHands.HandLandmark.RING_FINGER_PIP].y):
        count += 1
    
    #Meñique
    if (handLms.landmark[mpHands.HandLandmark.PINKY_TIP].y < handLms.landmark[mpHands.HandLandmark.PINKY_PIP].y):
        count += 1
        
    arr[2] = count
    
    #Si es la mano izquierda y no enseñamos ningún dedo
    if(hand == "Left"):
        if(count == 0):
            x = int(handLms.landmark[mpHands.HandLandmark.MIDDLE_FINGER_PIP].x * w)
            y = int(handLms.landmark[mpHands.HandLandmark.MIDDLE_FINGER_PIP].y * h)
            if(x >= 640):
                x = 639
                
            if (x < 0):
                x = 0
            
            if(y >= 480):
                y = 479    
                
            if(y < 0):
                y = 0
                
            arr[0] = x
            arr[1] = y
        
        else:
            x = int(handLms.landmark[mpHands.HandLandmark.INDEX_FINGER_TIP].x * w)
            y = int(handLms.landmark[mpHands.HandLandmark.INDEX_FINGER_TIP].y * h)
            if(x >= 640):
                x = 639
                
            if (x < 0):
                x = 0
            
            if(y >= 480):
                y = 479    
                
            if(y < 0):
                y = 0
                
            arr[0] = x
            arr[1] = y
            
    return arr

def ObtainColor(x,y,h,w):
    R = 255 - (x / w) * 255
    G = (255 - (x / w) * 255) * ((x / w) * 255)
    B = (x / w) * 255
    return(R, G, B)

R = 0
G = 0
B = 255

while(True):      
    # fotograma a fotograma
    ret, frame = vid.read()
    frame = cv2.flip(frame,1)
    
    x = 0
    y = 0
    h, w, c = frame.shape
    
    
    imageRGB = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(imageRGB)

    if results.multi_hand_landmarks:
        showing = True
        for handLms in results.multi_hand_landmarks: # working with each hand
            countedArr = countFingers(handLms, results.multi_handedness[0].classification[0].label)
            x = int(countedArr[0])
            y = int(countedArr[1])
            count = int(countedArr[2])
            if(results.multi_handedness[0].classification[0].label == "Right"):
                drawn[x, y] = 1             
                thicknes[x, y] = count

            else:
                if(count == 0):
                    if (x > 20 and x < 619) and (y > 20 and y < 439):
                        drawn[x-20:x + 20, y-20 : y + 20] = 0
                    else:
                        drawn[x, y] = 0
                else:
                    R, G, B = ObtainColor(x,y,h,w)

                    
    #Miramos que celdas se han dibujado
    arr = np.transpose((drawn > 0).nonzero())

    #Pintamos donde se haya tocado
    for i in range(len(arr)):
        cv2.circle(frame, (arr[i][0], arr[i][1]), 4 * int(thicknes[arr[i][0], arr[i][1]]) , (R, G, B), cv2.FILLED)
    
    cv2.imshow("Pictionary", frame)
    cv2.waitKey(1)

    # Detenemos pulsado ESC
    if cv2.waitKey(20) == 27:
        break
  
# Libera el objeto de captura
vid.release()
# Destruye ventanas
cv2.destroyAllWindows()